<a href="https://colab.research.google.com/github/kjw3757/Guide-Book/blob/main/Recommendation_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np 

df=pd.read_csv('/content/test-with-difficulty-last.csv')


In [ ]:
df.drop(df.columns[6:26], axis=1, inplace=True)
df_new = df[['Title', 'Author', 'Category', 'Publisher', 'Difficulty', 'Content']].copy()
df_new.columns = ['Title', 'Author', 'Category', 'Publisher', 'Difficulty', 'Content']


In [ ]:
df.head(5)

,Title,Author,Category,Publisher,Difficulty,Content
0,오만과 편견 (한글+영문판),제인 오스틴,고전,더클래식,## Book: 오만과 편견 (한글+영문판) by 제인 오스틴\n\n### Diff...,**Pride and Prejudice (Korean and English Edit...
1,코스모스,칼 세이건,과학,사이언스북,NaN,NaN
2,생각에 관한 생각,대니얼 카너먼,경제경영,김영사,"Here is the output for the book ""생각에 관한 생각"" by...","Sure, I can do that. Here is the information a..."
3,100가지 동물로 읽는 세계사,사이먼 반즈,역사,현대지성,"Sure, I can help you with that. Here is the in...","Sure, I can do that. Here is the information f..."
4,1984 (한글+영문),조지 오웰,소설,더클래식,"Sure, I can help you with that. Here is the di...","Sure, here is the information about the book 1..."


Plot description based Recommender

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorizer 객체를 정의 'the', 'a'와 같은 영어 불용어를 제거
tfidf = TfidfVectorizer(stop_words='english')

# NaN 값을 빈 문자열로 대체
df['Content'] = df['Content'].fillna('')

# 데이터를 fitting하고 변환하여 필요한 TF-IDF 행렬을 생성
tfidf_matrix = tfidf.fit_transform(df['Content'])

# tfidf_matrix의 형태(shape)를 출력
tfidf_matrix.shape

(46, 2154)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

# 코사인유사도 함수 생성
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
# 인덱스와 책 제목의 역 매핑을 생성
indices = pd.Series(df.index, index=df['Title']).drop_duplicates()

In [ ]:
# 책 제목을 입력으로 받아 가장 유사한 책을 출력하는 함수를 정의
def get_recommendations(title, cosine_sim=cosine_sim):
    
    idx = indices[title]

    # 해당 책과 다른 책들 간의 유사도 점수를 가져오기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도 점수를 기준으로 책들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 상위 10개의 유사한 책의 점수를 가져오기
    sim_scores = sim_scores[1:11]

    # 책의 인덱스를 가져오기
    book_indices = [i[0] for i in sim_scores]

    # 상위 10개의 유사한 책 반환
    return df['Title'].iloc[book_indices]

In [ ]:
get_recommendations('성냥팔이 소녀')

44               인어공주
31              어린 왕자
21           수레바퀴 아래서
12            이상 - 날개
26         이토록 평범한 미래
10             오만과 편견
39          아홉살 마음 사전
37    괴롭히려고 한 건 아니야! 
7              언어의 무게
19              우주의 집
Name: Title, dtype: object

Difficulty and GenresBased Recommender

In [ ]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['Content']
for feature in features:
    df[feature] = df[feature].apply(literal_eval)

SyntaxError: ignored

In [ ]:
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_author(x):
    for i in x:
        if i['job'] == 'Author':
            return i['name']
    return np.nan

In [ ]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [ ]:
# Define new director, cast, genres and keywords features that are in a suitable form.
df['Author'] = df['crew'].apply(get_author)

features = ['Category']
for feature in features:
    df[feature] = df[feature].apply(get_list)

In [ ]:
# Print the new features of the first 3 films
df[['Title', 'Author', 'Publisher', 'Category']].head(3)

In [ ]:
from ast import literal_eval
from sklearn.metrics.pairwise import cosine_similarity

def recommend_books(file_path, category, difficulty):
    # 엑셀 파일 읽기
    df = pd.read_excel(file_path)
    
    # literal_eval을 사용하여 열 데이터 변환
    df['Category'] = df['Category'].apply(literal_eval)
    df['Difficulty'] = df['Difficulty'].apply(literal_eval)
    
    # 입력된 카테고리와 난이도에 해당하는 책 필터링
    filtered_books = df[(df['Category'] == category) & (df['Difficulty'] == difficulty)]
    
    # TF-IDF 벡터화
    vectorizer = TfidfVectorizer()
    book_vectors = vectorizer.fit_transform(filtered_books['Content'])
    
    # 코사인 유사도 계산
    similarity_matrix = cosine_similarity(book_vectors)
    
    # 입력된 책과 유사도가 가장 높은 책 추출
    book_index = similarity_matrix[0].argsort()[-2:-12:-1]  # 자기 자신은 제외하고 상위 10개 추출
    recommended_books = filtered_books.iloc[book_index]
    
    return recommended_books

# 추천 알고리즘 테스트
file_path = 'books.xlsx'
category = 'Mystery'
difficulty = 'Intermediate'
recommended_books = recommend_books(file_path, category, difficulty)
print(recommended_books[['Title', 'Author', 'Publisher']])